In [1]:
rd=sc.parallelize([1,2,4,5,34,1,32,4,34,2,1,3])

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20200912175043-0000
KERNEL_ID = f2c1255d-d97e-43de-b85f-43997623387e


In [60]:
sum1=rd.sum()
n=rd.count()
mean=sum1/n
print(mean)


10.25


In [61]:
sortedrd=rd.sortBy(lambda x: x).collect()
print(sortedrd)

[1, 1, 1, 2, 2, 3, 4, 4, 5, 32, 34, 34]


In [62]:
n=rd.count()
print(n)
if(n%2==0):
    #print("even case")
    i1=n//2
    i2=(n//2)-1
    #print(i1,i2)
    #print(sortedrd[i1],sortedrd[i2])
    median=(sortedrd[i1]+sortedrd[i2])/2
    print(median)
else:
    #print("odd case")
    median=(n-1)//2
    print(sortedrd[median])

12
3.5


### Median is telling you an outlier resistant average of distances and standard deviation tells you about the range where values are spread around.

In [7]:
rdd=sc.parallelize([34,1,23,4,3,3,12,4,3,1])
sum1=rdd.sum()
n=rdd.count()
mean=sum1/n
print(mean)


8.8


In [8]:
from math import sqrt
sd=sqrt(rdd.map(lambda x:pow(x-mean,2)).sum()/n) #standard deviation
print(sd)

10.562196741208714


### Skewness basically tells you how asymmetric the data is spread around the mean. 
Note that incase we have a long tailed, thin left skew, and a short tailed,
thick right skew, they might cancel out if you obtain a value close to zero for
the skewness measure. 

In [9]:
n=float(n)
skew=rdd.map(lambda x:pow(x-mean,3)).sum()/(n*pow(sd,3)) #skewness
print(skew)

1.4549069329913997


### Kurtosis basically tells you something about the shape of your data when plotted using a histogram.
It tells you about the outlier content or in other words, the length of the tails of the histogram.
The higher the kurtosis measure is, the more outliers are present and he longer the tails of the distribution in the histogram are. 

In [5]:
kurtosis=rdd.map(lambda x:pow(x-mean,4)).sum()/(n*pow(sd,4)) #Kurtosis
print(kurtosis)

3.663124005193276


### Covariance and correlation tell us how two columns are interacting with each other.

In [36]:
X=sc.parallelize([1,2,3,4,5,6,7,8,9,10])
Y=sc.parallelize([7,6,5,4,5,6,7,8,9,10])
meanX=X.sum()/X.count()
meanY=Y.sum()/Y.count()
print(meanX, meanY)

5.5 6.7


In [37]:
XY=X.zip(Y)
covXY=XY.map(lambda x: (x[0]-meanX)*(x[1]-meanY)).sum()/XY.count()
print(covXY)
#XY.take(10)

3.65


In [38]:
from math import sqrt
n=XY.count()
sdX=sqrt(X.map(lambda x:pow(x-meanX,2)).sum()/n) #standard deviation
sdY=sqrt(Y.map(lambda y:pow(y-meanY,2)).sum()/n) #standard deviation
print(sdX,sdY)

2.8722813232690143 1.7916472867168916


In [39]:
corrXY=covXY/(sdX*sdY) #corelation
corrXY

0.709272912083725

## Corr=1 -> positive dependency
## Corr=0 -> no dependency means a lot of information from both datasets
## Corr=-1 -> negative dependency

In [35]:
# Covariance Matrix
import random
from pyspark.mllib.stat import Statistics as st
col1=sc.parallelize(range(100))
col2=sc.parallelize(range(100,200))
col3=sc.parallelize(list(reversed(range(100))))
col4=sc.parallelize(random.sample(range(100),100))
data=col1.zip(col2).zip(col3).zip(col4)  #zip of all columns forms nested tuples
data=data.map(lambda x: [x[0][0][0],x[0][0][1],x[0][1],x[1]]) #break of nested tuples to list
print(data.take(10))
st.corr(data)

[[0, 100, 99, 57], [1, 101, 98, 81], [2, 102, 97, 68], [3, 103, 96, 70], [4, 104, 95, 35], [5, 105, 94, 98], [6, 106, 93, 0], [7, 107, 92, 13], [8, 108, 91, 59], [9, 109, 90, 16]]


array([[ 1.        ,  1.        , -1.        , -0.08627663],
       [ 1.        ,  1.        , -1.        , -0.08627663],
       [-1.        , -1.        ,  1.        ,  0.08627663],
       [-0.08627663, -0.08627663,  0.08627663,  1.        ]])